In [1]:
versioninfo()
# Comment
# https://jump.dev/JuMP.jl/stable/tutorials/linear/diet/

Julia Version 1.6.2
Commit 1b93d53fc4 (2021-07-14 15:36 UTC)
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, skylake)


In [5]:
using JuMP

In [6]:
using Pkg

In [7]:
Pkg.add("DataFrames")

    Updating registry at `C:\Users\DSEP0001\.julia\registries\General`
    Updating git-repo `https://github.com/JuliaRegistries/General.git`
   Resolving package versions...
    Updating `C:\Users\DSEP0001\.julia\environments\v1.6\Project.toml`
  [a93c6f00] + DataFrames v1.3.4
  No Changes to `C:\Users\DSEP0001\.julia\environments\v1.6\Manifest.toml`
Precompiling project...
  ✓ Libmount_jll
  ✓ Xorg_libXdmcp_jll
  ✓ Xorg_libXau_jll
  ✓ Xorg_libpthread_stubs_jll
  ✓ Libgpg_error_jll
  ✓ JpegTurbo_jll
  ✓ LERC_jll
  ✓ Wayland_protocols_jll
  ✓ SCS_GPU_jll
  ✓ Xorg_xtrans_jll
  ✓ Wayland_jll
  ✓ Zstd_jll
  ✓ Libgcrypt_jll
  ✓ Glib_jll
  ✓ XSLT_jll
  ✓ Libtiff_jll
  ✓ Xorg_libxcb_jll
  ✓ SCS
  ✓ Xorg_libX11_jll
  ✓ Xorg_xcb_util_jll
  ✓ Xorg_libXrender_jll
  ✓ Xorg_libXfixes_jll
  ✓ Xorg_libxkbfile_jll
  ✓ Xorg_libXext_jll
  ✓ Xorg_xcb_util_keysyms_jll
  ✓ Xorg_xcb_util_renderutil_jll
  ✓ Xorg_xcb_util_image_jll
  ✓ Xorg_libXcursor_jll
  ✓ Xorg_xcb_util_wm_jll
  ✓ Xorg_xkbcomp_jll
  ✓ Lib

In [8]:
Pkg.add("HiGHS")

   Resolving package versions...
   Installed HiGHS_jll ─ v1.2.0+2
   Installed HiGHS ───── v0.3.2
    Updating `C:\Users\DSEP0001\.julia\environments\v1.6\Project.toml`
  [87dc4568] + HiGHS v0.3.2
    Updating `C:\Users\DSEP0001\.julia\environments\v1.6\Manifest.toml`
  [87dc4568] + HiGHS v0.3.2
  [8fd58aa0] + HiGHS_jll v1.2.0+2
Precompiling project...
  ✓ HiGHS_jll
  ✓ HiGHS
  2 dependencies successfully precompiled in 15 seconds (355 already precompiled, 4 skipped during auto due to previous errors)


In [9]:
using DataFrames
using HiGHS

In [10]:
foods = DataFrames.DataFrame(
    [
        "hamburger" 2.49 410 24 26 730
        "chicken" 2.89 420 32 10 1190
        "hot dog" 1.50 560 20 32 1800
        "fries" 1.89 380 4 19 270
        "macaroni" 2.09 320 12 10 930
        "pizza" 1.99 320 15 12 820
        "salad" 2.49 320 31 12 1230
        "milk" 0.89 100 8 2.5 125
        "ice cream" 1.59 330 8 10 180
    ],
    ["name", "cost", "calories", "protein", "fat", "sodium"],
)

,name,cost,calories,protein,fat,sodium
,Any,Any,Any,Any,Any,Any
1,hamburger,2.49,410,24,26,730
2,chicken,2.89,420,32,10,1190
3,hot dog,1.5,560,20,32,1800
4,fries,1.89,380,4,19,270
5,macaroni,2.09,320,12,10,930
6,pizza,1.99,320,15,12,820
7,salad,2.49,320,31,12,1230
8,milk,0.89,100,8,2.5,125
9,ice cream,1.59,330,8,10,180


In [11]:
limits = DataFrames.DataFrame(
    [
        "calories" 1800 2200
        "protein" 91 Inf
        "fat" 0 65
        "sodium" 0 1779
    ],
    ["name", "min", "max"],
)

,name,min,max
,Any,Any,Any
1,calories,1800,2200
2,protein,91,Inf
3,fat,0,65
4,sodium,0,1779


In [12]:
model = Model(HiGHS.Optimizer)

A JuMP Model
Feasibility problem with:
Variables: 0
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: HiGHS

In [13]:
@variable(model, x[foods.name] >= 0)

1-dimensional DenseAxisArray{VariableRef,1,...} with index sets:
    Dimension 1, Any["hamburger", "chicken", "hot dog", "fries", "macaroni", "pizza", "salad", "milk", "ice cream"]
And data, a 9-element Vector{VariableRef}:
 x[hamburger]
 x[chicken]
 x[hot dog]
 x[fries]
 x[macaroni]
 x[pizza]
 x[salad]
 x[milk]
 x[ice cream]

In [14]:
@objective(
    model,
    Min,
    sum(food["cost"] * x[food["name"]] for food in eachrow(foods)),
)

2.49 x[hamburger] + 2.89 x[chicken] + 1.5 x[hot dog] + 1.89 x[fries] + 2.09 x[macaroni] + 1.99 x[pizza] + 2.49 x[salad] + 0.89 x[milk] + 1.59 x[ice cream]

In [15]:
for limit in eachrow(limits)
    intake = @expression(
        model,
        sum(food[limit["name"]] * x[food["name"]] for food in eachrow(foods)),
    )
    @constraint(model, limit.min <= intake <= limit.max)
end

In [16]:
print(model)

In [17]:
optimize!(model)


solution_summary(model)

Presolving model
4 rows, 9 cols, 36 nonzeros
4 rows, 9 cols, 36 nonzeros
Presolve : Reductions: rows 4(-0); columns 9(-0); elements 36(-0)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 2(814) 0s
          6     1.1828861111e+01 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 6
Objective value     :  1.1828861111e+01
HiGHS run time      :          0.01


* Solver : HiGHS

* Status
  Termination status : OPTIMAL
  Primal status      : FEASIBLE_POINT
  Dual status        : FEASIBLE_POINT
  Message from the solver:
  "kOptimal"

* Candidate solution
  Objective value      : 11.828861111111111
  Objective bound      : 0.0
  Dual objective value : 11.828861111111113

* Work counters
  Solve time (sec)   : 0.01296


In [18]:
for food in foods.name
    println(food, " = ", value(x[food]))
end

hamburger = 0.6045138888888888
chicken = 0.0
hot dog = 0.0
fries = 0.0
macaroni = 0.0
pizza = 0.0
salad = 0.0
milk = 6.970138888888889
ice cream = 2.5913194444444447


In [19]:
# additional constraint
@constraint(model, x["milk"] + x["ice cream"] <= 6)

x[milk] + x[ice cream] <= 6.0

In [20]:
optimize!(model)


solution_summary(model)

Solving LP without presolve or with basis
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     1.1828875782e+01 Pr: 1(56.9833) 0s
          0     1.1828875782e+01 0s
Model   status      : Infeasible
Objective value     :  1.1828861111e+01
HiGHS run time      :          0.01


LoadError: MethodError: [0mCannot `convert` an object of type [92mMissing[39m[0m to an object of type [91mFloat64[39m
[0mClosest candidates are:
[0m  convert(::Type{T}, [91m::Base.TwicePrecision[39m) where T<:Number at twiceprecision.jl:250
[0m  convert(::Type{T}, [91m::AbstractChar[39m) where T<:Number at char.jl:180
[0m  convert(::Type{T}, [91m::CartesianIndex{1}[39m) where T<:Number at multidimensional.jl:136
[0m  ...

In [22]:
# Another example
# https://jump.dev/JuMP.jl/stable/tutorials/linear/facility_location/

In [23]:
Pkg.add("LinearAlgebra")
Pkg.add("Plots")
Pkg.add("Random")
using LinearAlgebra
using Plots
using Random

   Resolving package versions...
  No Changes to `C:\Users\DSEP0001\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\DSEP0001\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\DSEP0001\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\DSEP0001\.julia\environments\v1.6\Manifest.toml`
   Resolving package versions...
    Updating `C:\Users\DSEP0001\.julia\environments\v1.6\Project.toml`
  [9a3f8284] + Random
  No Changes to `C:\Users\DSEP0001\.julia\environments\v1.6\Manifest.toml`
